# Model Training: XGBoost Walk-Forward Validation

Train XGBoost model with walk-forward validation and hyperparameter optimization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_acquisition import DataAcquisition
from feature_engineering import FeatureEngineer
from ml_model import CryptoMLModel

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Load data and features
data_acq = DataAcquisition('config.yaml')
dataset = data_acq.fetch_full_dataset()

engineer = FeatureEngineer('config.yaml')
features = engineer.engineer_all_features(dataset['prices'], dataset['events'])
target = engineer.create_target_variable(dataset['prices'])

print(f"Features shape: {features.shape}")
print(f"Target shape: {target.shape}")

## 1. Walk-Forward Validation (Without Hyperparameter Optimization)

In [ ]:
# Train without hyperparameter optimization (for speed)
model = CryptoMLModel('config.yaml')

print("Running walk-forward validation...")
ml_results = model.walk_forward_validation(
    features,
    target,
    optimize_hyperparams=False
)

print(f"\nOverall Sharpe Ratio: {ml_results['overall_sharpe']:.4f}")
print(f"Overall MSE: {ml_results['overall_mse']:.6f}")

print("\nPer-Fold Results:")
for fold in ml_results['fold_results']:
    print(f"  Fold {fold['fold']}: Sharpe={fold['sharpe']:.4f}, MSE={fold['mse']:.6f}, Period={fold['test_period']}")

## 2. Learning Curves: Sharpe Ratio Over Folds

In [ ]:
fold_df = pd.DataFrame(ml_results['fold_results'])

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Sharpe ratio
axes[0].plot(fold_df['fold'], fold_df['sharpe'], marker='o', linewidth=2, markersize=8, color='darkblue')
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5, label='Zero')
axes[0].axhline(ml_results['overall_sharpe'], color='green', linestyle='--', alpha=0.7, label='Overall Sharpe')
axes[0].set_ylabel('Sharpe Ratio', fontsize=12)
axes[0].set_title('Walk-Forward Validation: Sharpe Ratio per Fold', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# MSE
axes[1].plot(fold_df['fold'], fold_df['mse'], marker='s', linewidth=2, markersize=8, color='darkred')
axes[1].axhline(ml_results['overall_mse'], color='green', linestyle='--', alpha=0.7, label='Overall MSE')
axes[1].set_xlabel('Fold', fontsize=12)
axes[1].set_ylabel('MSE', fontsize=12)
axes[1].set_title('Walk-Forward Validation: MSE per Fold', fontsize=14, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nFold Statistics:")
print(f"  Mean Sharpe: {fold_df['sharpe'].mean():.4f}")
print(f"  Std Sharpe: {fold_df['sharpe'].std():.4f}")
print(f"  Min Sharpe: {fold_df['sharpe'].min():.4f}")
print(f"  Max Sharpe: {fold_df['sharpe'].max():.4f}")

## 3. Predictions vs Actuals

In [ ]:
results_df = ml_results['results_df']

# Scatter plot
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(results_df['actual'], results_df['predicted'], alpha=0.3, s=20)
ax.plot([-0.1, 0.1], [-0.1, 0.1], 'r--', linewidth=2, label='Perfect Prediction')
ax.set_xlabel('Actual Returns', fontsize=12)
ax.set_ylabel('Predicted Returns', fontsize=12)
ax.set_title('Predictions vs Actuals', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Correlation
corr = np.corrcoef(results_df['actual'], results_df['predicted'])[0, 1]
print(f"\nCorrelation: {corr:.4f}")

## 4. Prediction Errors Over Time

In [ ]:
results_df['error'] = results_df['predicted'] - results_df['actual']
results_df['abs_error'] = np.abs(results_df['error'])

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Errors
axes[0].plot(results_df.index, results_df['error'], linewidth=1, alpha=0.7, color='darkred')
axes[0].axhline(0, color='black', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Prediction Error', fontsize=12)
axes[0].set_title('Prediction Errors Over Time', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Absolute errors
axes[1].plot(results_df.index, results_df['abs_error'], linewidth=1, alpha=0.7, color='darkorange')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Absolute Error', fontsize=12)
axes[1].set_title('Absolute Prediction Errors Over Time', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nError Statistics:")
print(f"  Mean Error: {results_df['error'].mean():.6f}")
print(f"  Mean Absolute Error: {results_df['abs_error'].mean():.6f}")
print(f"  RMSE: {np.sqrt((results_df['error']**2).mean()):.6f}")

## 5. Hyperparameter Optimization (Optional - Long Runtime)

In [ ]:
# WARNING: This cell can take 30-60 minutes to run
# Uncomment to run hyperparameter optimization

# print("Running hyperparameter optimization (this will take a while)...")
# ml_results_optimized = model.walk_forward_validation(
#     features,
#     target,
#     optimize_hyperparams=True
# )

# print(f"\nOptimized Overall Sharpe Ratio: {ml_results_optimized['overall_sharpe']:.4f}")
# print(f"Optimized Overall MSE: {ml_results_optimized['overall_mse']:.6f}")

# print("\nComparison:")
# print(f"  Default Sharpe: {ml_results['overall_sharpe']:.4f}")
# print(f"  Optimized Sharpe: {ml_results_optimized['overall_sharpe']:.4f}")
# print(f"  Improvement: {ml_results_optimized['overall_sharpe'] - ml_results['overall_sharpe']:.4f}")

## 6. Feature Importance

In [ ]:
# Train a single model for feature importance analysis
common_idx = features.index.intersection(target.index)
X = features.loc[common_idx].values
y = target.loc[common_idx].values

# Use default parameters
params = {
    'n_estimators': 100,
    'max_depth': 5,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

trained_model = model.train_xgboost(X, y, None, None, params)
importance = model.get_feature_importance(trained_model)

# Map indices to feature names
importance.index = features.columns
importance = importance.sort_values(ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
importance.plot(kind='barh', ax=ax, color='steelblue', alpha=0.8)
ax.set_xlabel('Importance (Gain)', fontsize=12)
ax.set_title('XGBoost Feature Importance', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 5 Features by Importance:")
for feature, imp in importance.head(5).items():
    print(f"  {feature}: {imp:.4f}")

## 7. Time Series of Predictions

In [ ]:
# Plot predictions and actuals over time
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(results_df.index, results_df['actual'], label='Actual Returns', linewidth=1, alpha=0.7)
ax.plot(results_df.index, results_df['predicted'], label='Predicted Returns', linewidth=1, alpha=0.7)
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Log Returns', fontsize=12)
ax.set_title('Actual vs Predicted Returns Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Directional Accuracy

In [ ]:
# Calculate directional accuracy
results_df['actual_sign'] = np.sign(results_df['actual'])
results_df['predicted_sign'] = np.sign(results_df['predicted'])
results_df['correct_direction'] = results_df['actual_sign'] == results_df['predicted_sign']

directional_accuracy = results_df['correct_direction'].mean()

print(f"\nDirectional Accuracy: {directional_accuracy:.2%}")
print("\nBreakdown:")
print(f"  Total predictions: {len(results_df)}")
print(f"  Correct direction: {results_df['correct_direction'].sum()}")
print(f"  Incorrect direction: {(~results_df['correct_direction']).sum()}")

# Plot cumulative directional accuracy
cumulative_accuracy = results_df['correct_direction'].cumsum() / (np.arange(len(results_df)) + 1)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(results_df.index, cumulative_accuracy, linewidth=2, color='darkgreen')
ax.axhline(0.5, color='red', linestyle='--', alpha=0.5, label='Random (50%)')
ax.axhline(directional_accuracy, color='blue', linestyle='--', alpha=0.7, label=f'Overall ({directional_accuracy:.1%})')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Directional Accuracy', fontsize=12)
ax.set_title('Cumulative Directional Accuracy Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()